# Générateur de Noms de Domaine avec LLM
## Expérimentations et Évaluations Systématiques

Ce notebook présente le développement itératif d'un système de génération de noms de domaine utilisant des modèles de langage, avec un focus sur l'évaluation automatisée et la découverte de cas limites.

## 1. Configuration et Imports

In [1]:
import sys
import os
sys.path.append("..")
from datetime import datetime

# Import des modules du projet
from src.creation_dataset import CreationDataset
from src.utils import charger_json


print(f"Date de l'expérience: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

Date de l'expérience: 2025-08-02 00:06:19


## 2. Création du Dataset Synthétique

Nous commençons par créer un dataset synthétique diversifié pour l'évaluation.

In [2]:
# Initialiser le créateur de dataset
creation_dataset = CreationDataset()

# Créer le dataset complet
print("Création du dataset synthétique...")
chemin_dataset = creation_dataset.sauvegarder_dataset("dataset_experimental.json")

# Charger et analyser le dataset créé
dataset_complet = charger_json(chemin_dataset)

print(f"Dataset créé avec succès:")
print(f"   - Exemples d'entraînement: {len(dataset_complet['entrainement'])}")
print(f"   - Cas limites: {len(dataset_complet['cas_limites'])}")

# Visualisation de la distribution des types de business
descriptions = [ex['description_business'] for ex in dataset_complet['entrainement']]
print(f"\nExemples de descriptions générées:")
for i, desc in enumerate(descriptions[:5]):
    print(f"   {i+1}. {desc}")

INFO:src.utils:Génération d'un dataset de 800 exemples...
INFO:src.utils:Généré 100/800 exemples
INFO:src.utils:Généré 200/800 exemples
INFO:src.utils:Généré 300/800 exemples
INFO:src.utils:Généré 400/800 exemples
INFO:src.utils:Généré 500/800 exemples
INFO:src.utils:Généré 600/800 exemples
INFO:src.utils:Généré 700/800 exemples
INFO:src.utils:Généré 800/800 exemples
INFO:src.utils:Données sauvegardées dans ../data/dataset_experimental.json
INFO:src.utils:Dataset complet sauvegardé: ../data/dataset_experimental.json
INFO:src.utils:Données chargées depuis ../data/dataset_experimental.json


Création du dataset synthétique...
Dataset créé avec succès:
   - Exemples d'entraînement: 800
   - Cas limites: 14

Exemples de descriptions générées:
   1. pratique spécialisé en kinésithérapie situé hôpital
   2. café végétarien dans quartier résidentiel
   3. vente vêtements dans centre commercial
   4. clinique médecine générale dans domicile
   5. Nouveau bistro végétarien à bord de mer
